# Intelli-ODM (Intelligent On-Demand Merchandising) Demo

This notebook demonstrates the complete workflow of the Intelli-ODM system, from product analysis to procurement recommendations.

## Overview

The Intelli-ODM system uses AI agents to:
1. **Data Ingestion**: Process and validate product data
2. **Attribute Analysis**: Extract product attributes and find comparable items
3. **Demand Forecasting**: Predict future demand using multiple approaches
4. **Procurement Optimization**: Generate optimal procurement recommendations

Let's walk through each step with real examples.

In [ ]:
# Setup and imports
import sys
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Add the parent directory to Python path
sys.path.append('..')

# Import our modules
from config.settings import Settings
from shared_knowledge_base import SharedKnowledgeBase
from utils.llm_client import LLMClientFactory
from agents.orchestrator_agent import OrchestratorAgent

print("✅ All imports successful!")

## 1. Configuration and Setup

First, let's set up our LLM client and knowledge base. This example supports both Ollama (local) and OpenAI.

In [ ]:
# Load configuration
settings = Settings()

print(f"🔧 Using LLM Provider: {settings.llm_provider}")

# Create LLM client
if settings.llm_provider == "ollama":
    llm_config = {
        "provider": "ollama",
        "base_url": settings.ollama_base_url,
        "model": settings.ollama_model,
        "timeout": 300
    }
elif settings.llm_provider == "openai":
    if not settings.openai_api_key:
        print("❌ OpenAI API key not found. Please set OPENAI_API_KEY environment variable.")
        print("   Falling back to Ollama...")
        llm_config = {"provider": "ollama", "base_url": "http://localhost:11434", "model": "llama3:8b"}
    else:
        llm_config = {
            "provider": "openai",
            "api_key": settings.openai_api_key,
            "model": settings.openai_model
        }

try:
    llm_client = LLMClientFactory.create_client(llm_config)
    print(f"✅ LLM client created successfully")
    
    # Test LLM availability
    if llm_client.is_available():
        print(f"🟢 LLM service is available and ready")
    else:
        print(f"🟡 LLM service not available - some features will use fallbacks")
        
except Exception as e:
    print(f"❌ Failed to create LLM client: {e}")
    print("   Please ensure Ollama is running or OpenAI API key is set")

In [ ]:
# Initialize knowledge base
try:
    kb = SharedKnowledgeBase(persist_directory="../data/chroma_db")
    print(f"✅ Knowledge base initialized")
    print(f"📊 Current products in knowledge base: {kb.get_collection_size()}")
except Exception as e:
    print(f"❌ Failed to initialize knowledge base: {e}")

# Initialize orchestrator
try:
    orchestrator = OrchestratorAgent(llm_client, kb)
    print(f"✅ Orchestrator agent initialized successfully")
except Exception as e:
    print(f"❌ Failed to initialize orchestrator: {e}")

## 2. Sample Product Descriptions

Let's define some sample product descriptions to analyze. These represent real-world product listings that might be found in e-commerce catalogs.

In [ ]:
# Sample product descriptions
sample_products = [
    "Classic white cotton t-shirt with crew neck, short sleeves, regular fit, perfect for casual wear. Made from 100% premium cotton.",
    "Navy blue denim jeans, slim fit, straight leg, button fly closure. High-quality denim material with slight stretch for comfort.",
    "Elegant black cocktail dress, sleeveless design with V-neckline, knee-length, made from premium chiffon fabric. Perfect for formal occasions.",
    "Red polo shirt for men, cotton blend fabric, collar with 3-button placket, short sleeves, athletic fit design.",
    "Women's floral print summer dress, midi length, cap sleeves, round neckline, lightweight crepe material. Ideal for spring and summer."
]

print(f"📦 Sample products to analyze: {len(sample_products)}")
for i, desc in enumerate(sample_products, 1):
    print(f"{i}. {desc[:60]}...")

## 3. Sample Inventory and Sales Data

Let's create some sample inventory levels and sales history to make our analysis more realistic.

In [ ]:
# Sample inventory data
inventory_data = {
    "product_1": 150,  # White t-shirt
    "product_2": 80,   # Navy jeans
    "product_3": 25,   # Black dress
    "product_4": 120,  # Red polo
    "product_5": 40    # Floral dress
}

# Generate sample sales history (last 30 days)
def generate_sales_history(product_count=5, days=30):
    import random
    sales_history = []
    
    base_date = datetime.now() - timedelta(days=days)
    
    for day in range(days):
        date = base_date + timedelta(days=day)
        
        for product_id in range(1, product_count + 1):
            # Generate realistic sales with some seasonality and randomness
            base_sales = {
                1: 15,  # T-shirt (popular)
                2: 8,   # Jeans (steady)
                3: 3,   # Dress (lower volume)
                4: 10,  # Polo (moderate)
                5: 5    # Floral dress (seasonal)
            }
            
            # Add some randomness and day-of-week effects
            weekday_factor = 1.3 if date.weekday() < 5 else 0.7  # Weekday vs weekend
            random_factor = random.uniform(0.7, 1.4)
            
            daily_sales = max(0, int(base_sales[product_id] * weekday_factor * random_factor))
            
            if daily_sales > 0:
                sales_history.append({
                    "product_id": f"product_{product_id}",
                    "date": date.strftime("%Y-%m-%d"),
                    "quantity": daily_sales,
                    "revenue": daily_sales * random.uniform(40, 80)  # Variable pricing
                })
    
    return sales_history

sales_history = generate_sales_history()

print(f"📈 Generated {len(sales_history)} sales records")
print(f"💰 Sample inventory levels: {inventory_data}")

# Show sales summary
sales_df = pd.DataFrame(sales_history)
sales_summary = sales_df.groupby('product_id').agg({
    'quantity': 'sum',
    'revenue': 'sum'
}).round(2)

print("\n📊 Sales Summary (Last 30 days):")
print(sales_summary)

## 4. Execute Complete Workflow

Now let's run the complete Intelli-ODM workflow using our orchestrator agent. This will execute all phases of the analysis.

In [ ]:
print("🚀 Starting complete Intelli-ODM workflow...\n")

# Execute the complete workflow
try:
    results = orchestrator.run_complete_workflow(
        product_descriptions=sample_products,
        inventory_data=inventory_data,
        sales_history=sales_history
    )
    
    if results['success']:
        print("✅ Workflow completed successfully!")
        print(f"📋 Session ID: {results['session_id']}")
        print(f"⭐ Confidence Score: {results['confidence_score']:.2f}")
        print(f"📊 Success Rate: {results['metrics']['success_rate']:.1%}")
    else:
        print("❌ Workflow failed:")
        print(f"Error: {results.get('error', 'Unknown error')}")
        
except Exception as e:
    print(f"❌ Workflow execution failed: {e}")
    results = None

## 5. Analyze Results

Let's examine the results from each phase of the analysis.

In [ ]:
if results and results['success']:
    print("📋 EXECUTIVE SUMMARY")
    print("=" * 50)
    print(results['executive_summary'])
    print()
    
    print("🔍 KEY INSIGHTS")
    print("=" * 50)
    for insight in results['key_insights']:
        print(f"• {insight}")
    print()
    
    print("📊 EXECUTION METRICS")
    print("=" * 50)
    metrics = results['metrics']
    print(f"Products Processed: {metrics['total_products_processed']}")
    print(f"Successful Analyses: {metrics['successful_analyses']}")
    print(f"Forecasts Generated: {metrics['forecasts_generated']}")
    print(f"Procurement Recommendations: {metrics['procurement_recommendations']}")
    print(f"Overall Success Rate: {metrics['success_rate']:.1%}")
    
    print("\n⏱️ Execution Times:")
    for phase, time_taken in metrics['execution_times'].items():
        print(f"  {phase}: {time_taken:.2f} seconds")
else:
    print("❌ No results to analyze")

## 6. Product Attribute Analysis

Let's examine the extracted product attributes and comparable products.

In [ ]:
if results and results['success']:
    attribute_results = results['detailed_results']['attribute_analysis']
    
    print("🏷️ PRODUCT ATTRIBUTES ANALYSIS")
    print("=" * 60)
    
    # Create a DataFrame for better visualization
    attributes_data = []
    
    for product_id, attrs in attribute_results.get('product_attributes', {}).items():
        attributes_data.append({
            'Product': product_id,
            'Category': attrs.get('category', 'Unknown'),
            'Material': attrs.get('material', 'Unknown'),
            'Color': attrs.get('color', 'Unknown'),
            'Style': attrs.get('style', 'Unknown'),
            'Target Gender': attrs.get('target_gender', 'Unknown'),
            'Confidence': f"{attrs.get('confidence', 0):.2f}"
        })
    
    if attributes_data:
        attrs_df = pd.DataFrame(attributes_data)
        print(attrs_df.to_string(index=False))
        
        # Show category distribution
        print("\n📊 Category Distribution:")
        category_counts = attrs_df['Category'].value_counts()
        for category, count in category_counts.items():
            print(f"  {category}: {count} products")
    else:
        print("No product attributes extracted")

## 7. Demand Forecasting Results

Let's visualize the demand forecasts for our products.

In [ ]:
if results and results['success']:
    forecast_results = results['detailed_results']['demand_forecasting']
    
    print("📈 DEMAND FORECASTING RESULTS")
    print("=" * 60)
    
    # Extract forecast data
    forecast_data = []
    
    for product_id, forecast in forecast_results.get('demand_forecasts', {}).items():
        if isinstance(forecast.get('forecast'), dict):
            forecast_qty = forecast['forecast'].get('quantity', 0)
            confidence = forecast['forecast'].get('confidence', 0)
            trend = forecast['forecast'].get('trend', {}).get('direction', 'stable')
        else:
            forecast_qty = forecast.get('forecast', 0)
            confidence = forecast.get('confidence', 0.5)
            trend = 'stable'
        
        forecast_data.append({
            'Product': product_id,
            'Forecast (30 days)': int(forecast_qty),
            'Trend': trend,
            'Confidence': f"{confidence:.2f}",
            'Current Inventory': inventory_data.get(product_id, 0)
        })
    
    if forecast_data:
        forecast_df = pd.DataFrame(forecast_data)
        print(forecast_df.to_string(index=False))
        
        # Calculate inventory coverage
        forecast_df['Coverage (days)'] = (
            forecast_df['Current Inventory'] / 
            (forecast_df['Forecast (30 days)'] / 30)
        ).round(1)
        
        print("\n📊 Inventory Coverage Analysis:")
        for _, row in forecast_df.iterrows():
            coverage = row['Coverage (days)']
            status = "🔴 Critical" if coverage < 15 else "🟡 Low" if coverage < 30 else "🟢 Adequate"
            print(f"  {row['Product']}: {coverage} days {status}")
    else:
        print("No demand forecasts available")
        
    # Visualize forecasts
    if forecast_data:
        plt.figure(figsize=(12, 6))
        
        products = [item['Product'] for item in forecast_data]
        forecasts = [item['Forecast (30 days)'] for item in forecast_data]
        inventory = [item['Current Inventory'] for item in forecast_data]
        
        x = range(len(products))
        width = 0.35
        
        plt.bar([i - width/2 for i in x], forecasts, width, label='Forecast (30 days)', alpha=0.8)
        plt.bar([i + width/2 for i in x], inventory, width, label='Current Inventory', alpha=0.8)
        
        plt.xlabel('Products')
        plt.ylabel('Quantity')
        plt.title('Demand Forecast vs Current Inventory')
        plt.xticks(x, products, rotation=45)
        plt.legend()
        plt.tight_layout()
        plt.show()

## 8. Procurement Recommendations

Finally, let's examine the procurement optimization results and recommendations.

In [ ]:
if results and results['success']:
    procurement_results = results['detailed_results']['procurement_optimization']
    
    print("🛒 PROCUREMENT RECOMMENDATIONS")
    print("=" * 60)
    
    recommendations = results['recommendations']['procurement']
    
    if recommendations:
        # Create recommendations DataFrame
        rec_data = []
        
        for rec in recommendations:
            rec_data.append({
                'Product': rec['product_id'],
                'Action': rec['action'],
                'Quantity': rec['quantity'],
                'Cost': f"${rec['estimated_cost']:,.0f}",
                'Priority': rec['priority'],
                'Timeline': rec['timeline'],
                'Supplier': rec['supplier']['name']
            })
        
        rec_df = pd.DataFrame(rec_data)
        print(rec_df.to_string(index=False))
        
        # Summary statistics
        total_investment = sum(rec['estimated_cost'] for rec in recommendations)
        total_quantity = sum(rec['quantity'] for rec in recommendations)
        high_priority_count = len([r for r in recommendations if r['priority'] == 'High'])
        
        print(f"\n💰 PROCUREMENT SUMMARY")
        print(f"Total Investment Required: ${total_investment:,.0f}")
        print(f"Total Quantity to Procure: {total_quantity:,} units")
        print(f"High Priority Items: {high_priority_count}")
        print(f"Average Cost per Unit: ${total_investment/total_quantity:.2f}")
        
        # Priority breakdown
        priority_counts = {}
        for rec in recommendations:
            priority = rec['priority']
            priority_counts[priority] = priority_counts.get(priority, 0) + 1
        
        print(f"\n📊 Priority Breakdown:")
        for priority, count in priority_counts.items():
            print(f"  {priority}: {count} items")
            
        # Visualize procurement recommendations
        if len(recommendations) > 1:
            plt.figure(figsize=(10, 6))
            
            products = [rec['product_id'] for rec in recommendations]
            costs = [rec['estimated_cost'] for rec in recommendations]
            priorities = [rec['priority'] for rec in recommendations]
            
            # Color code by priority
            colors = {'High': 'red', 'Medium': 'orange', 'Low': 'green'}
            bar_colors = [colors.get(p, 'blue') for p in priorities]
            
            plt.bar(products, costs, color=bar_colors, alpha=0.7)
            plt.xlabel('Products')
            plt.ylabel('Procurement Cost ($)')
            plt.title('Procurement Recommendations by Priority')
            plt.xticks(rotation=45)
            
            # Create legend
            handles = [plt.Rectangle((0,0),1,1, color=colors[p], alpha=0.7) for p in colors.keys()]
            plt.legend(handles, colors.keys(), title='Priority')
            
            plt.tight_layout()
            plt.show()
    
    else:
        print("No procurement recommendations generated")
        
    # Strategic recommendations
    strategic_recs = results['recommendations'].get('strategic', [])
    if strategic_recs:
        print(f"\n🎯 STRATEGIC RECOMMENDATIONS")
        print("=" * 60)
        for i, rec in enumerate(strategic_recs, 1):
            print(f"{i}. {rec}")
else:
    print("❌ No results available for analysis")

## 9. Error Analysis and System Health

Let's examine any errors or warnings from the workflow execution.

In [ ]:
if results:
    print("🔍 SYSTEM HEALTH CHECK")
    print("=" * 50)
    
    errors = results.get('errors', [])
    if errors:
        print(f"⚠️ Errors encountered: {len(errors)}")
        for i, error in enumerate(errors, 1):
            print(f"{i}. {error}")
    else:
        print("✅ No errors encountered")
    
    # Check phase-specific errors
    if 'detailed_results' in results:
        phase_errors = {}
        for phase_name, phase_data in results['detailed_results'].items():
            phase_errors[phase_name] = len(phase_data.get('errors', []))
        
        print(f"\n📊 Phase Error Breakdown:")
        for phase, error_count in phase_errors.items():
            status = "✅" if error_count == 0 else "⚠️"
            print(f"  {status} {phase}: {error_count} errors")
    
    # System recommendations
    print(f"\n🔧 SYSTEM RECOMMENDATIONS")
    print("=" * 50)
    
    confidence = results.get('confidence_score', 0)
    success_rate = results.get('metrics', {}).get('success_rate', 0)
    
    if confidence < 0.7:
        print("• Consider improving data quality or LLM configuration")
    
    if success_rate < 0.8:
        print("• Check LLM service availability and network connectivity")
    
    if len(errors) > 0:
        print("• Review error logs and consider adjusting agent parameters")
    
    if confidence >= 0.8 and success_rate >= 0.8 and len(errors) == 0:
        print("✅ System is operating optimally!")
else:
    print("❌ No results available for health check")

## 10. Export Results

Let's export our results for further analysis or reporting.

In [ ]:
if results and results['success']:
    # Create export directory
    export_dir = "exports"
    os.makedirs(export_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Export complete results as JSON
    json_file = f"{export_dir}/intelli_odm_results_{timestamp}.json"
    with open(json_file, 'w') as f:
        json.dump(results, f, indent=2, default=str)
    print(f"✅ Results exported to: {json_file}")
    
    # Export recommendations as CSV
    if results['recommendations']['procurement']:
        rec_data = []
        for rec in results['recommendations']['procurement']:
            rec_data.append({
                'Product ID': rec['product_id'],
                'Action': rec['action'],
                'Quantity': rec['quantity'],
                'Estimated Cost': rec['estimated_cost'],
                'Priority': rec['priority'],
                'Timeline': rec['timeline'],
                'Supplier': rec['supplier']['name'],
                'Urgency Score': rec['urgency'],
                'Justification': rec['justification']
            })
        
        rec_df = pd.DataFrame(rec_data)
        csv_file = f"{export_dir}/procurement_recommendations_{timestamp}.csv"
        rec_df.to_csv(csv_file, index=False)
        print(f"✅ Recommendations exported to: {csv_file}")
    
    # Create summary report
    report_file = f"{export_dir}/executive_summary_{timestamp}.txt"
    with open(report_file, 'w') as f:
        f.write("INTELLI-ODM ANALYSIS REPORT\n")
        f.write("=" * 50 + "\n\n")
        f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Session ID: {results['session_id']}\n\n")
        
        f.write("EXECUTIVE SUMMARY\n")
        f.write("-" * 20 + "\n")
        f.write(f"{results['executive_summary']}\n\n")
        
        f.write("KEY METRICS\n")
        f.write("-" * 20 + "\n")
        metrics = results['metrics']
        f.write(f"Products Processed: {metrics['total_products_processed']}\n")
        f.write(f"Success Rate: {metrics['success_rate']:.1%}\n")
        f.write(f"Confidence Score: {results['confidence_score']:.2f}\n")
        f.write(f"Total Investment: ${results['recommendations']['procurement'][0]['estimated_cost'] if results['recommendations']['procurement'] else 0:,.0f}\n\n")
        
        f.write("KEY INSIGHTS\n")
        f.write("-" * 20 + "\n")
        for insight in results['key_insights']:
            f.write(f"• {insight}\n")
    
    print(f"✅ Summary report exported to: {report_file}")
    
    print(f"\n📁 All exports saved to '{export_dir}' directory")
else:
    print("❌ No results to export")

## Conclusion

This notebook demonstrated the complete Intelli-ODM workflow:

1. **Setup**: Configured LLM client and knowledge base
2. **Data Preparation**: Created sample products, inventory, and sales data
3. **Workflow Execution**: Ran the complete orchestrated analysis
4. **Result Analysis**: Examined attributes, forecasts, and recommendations
5. **Export**: Saved results for further use

The system successfully analyzed products, generated demand forecasts, and provided optimized procurement recommendations with confidence scoring and error handling.

### Next Steps

1. Integrate with real product catalogs and sales data
2. Fine-tune agent parameters based on your specific domain
3. Set up automated workflows for regular analysis
4. Customize the UI for your business needs

### Getting Help

- Check the documentation in the `docs/` folder
- Review agent configurations in `config/`
- Examine the knowledge base setup in `shared_knowledge_base.py`
- Test individual agents in isolation for debugging